In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

data = pd.read_csv('state_consumption.csv')
data

,Goa,Gujarat,Maharashtra,DNH,Rajasthan
0,12.8,319.5,428.6,18.6,234.1
1,13.7,316.7,419.6,18.2,240.2
2,12.6,301.9,395.8,16.7,239.8
3,13.0,313.2,411.1,17.6,239.1
4,12.9,320.7,408.6,18.6,240.4
...,...,...,...,...,...
454,12.3,312.0,421.6,18.2,219.8
455,8.8,289.7,369.2,17.4,203.2
456,8.8,297.9,395.5,18.3,212.6
457,8.8,296.8,395.5,18.7,225.5


In [2]:
model_filename = "my_table_model.h5"
if os.path.exists(model_filename):
    # Load the existing model
    model = tf.keras.models.load_model(model_filename)
    print("Existing model loaded.")
else:
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # No activation function for regression
])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    print("New model created.")


Existing model loaded.


In [3]:

# Split data into features and target
X = data[['Goa', 'Gujarat', 'Maharashtra', 'DNH', 'Rajasthan']]
data['total_electricity_consumption'] = data['Goa'] + data['Gujarat'] + \
    data['Maharashtra'] + data['DNH'] + data['Rajasthan']
y = data['total_electricity_consumption']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
data

,Goa,Gujarat,Maharashtra,DNH,Rajasthan,total_electricity_consumption
0,12.8,319.5,428.6,18.6,234.1,1013.6
1,13.7,316.7,419.6,18.2,240.2,1008.4
2,12.6,301.9,395.8,16.7,239.8,966.8
3,13.0,313.2,411.1,17.6,239.1,994.0
4,12.9,320.7,408.6,18.6,240.4,1001.2
...,...,...,...,...,...,...
454,12.3,312.0,421.6,18.2,219.8,983.9
455,8.8,289.7,369.2,17.4,203.2,888.3
456,8.8,297.9,395.5,18.3,212.6,933.1
457,8.8,296.8,395.5,18.7,225.5,945.3


In [4]:
# Check if model needs retraining
if not os.path.exists(model_filename):
    # Train the new model
    model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test))
else:
    # Retrain the existing model
    model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test))

Epoch 1/500
12/12 [==============================] - 0s 15ms/step - loss: 0.0494 - mae: 0.1952 - val_loss: 0.0156 - val_mae: 0.1209
Epoch 2/500
12/12 [==============================] - 0s 6ms/step - loss: 0.0197 - mae: 0.1256 - val_loss: 0.0355 - val_mae: 0.1863
Epoch 3/500
12/12 [==============================] - 0s 4ms/step - loss: 0.0445 - mae: 0.1813 - val_loss: 0.0566 - val_mae: 0.2364
Epoch 4/500
12/12 [==============================] - 0s 5ms/step - loss: 0.0570 - mae: 0.2114 - val_loss: 0.0591 - val_mae: 0.2418
Epoch 5/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0203 - mae: 0.1156 - val_loss: 0.0017 - val_mae: 0.0313
Epoch 6/500
12/12 [==============================] - 0s 5ms/step - loss: 0.0086 - mae: 0.0795 - val_loss: 0.0179 - val_mae: 0.1260
Epoch 7/500
12/12 [==============================] - 0s 4ms/step - loss: 0.0164 - mae: 0.1103 - val_loss: 0.0382 - val_mae: 0.1888
Epoch 8/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0221 - 

In [5]:
model.save(model_filename)
print("Model saved.")

Model saved.


c:\Users\rohit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
last_5_rows = data.tail(5)
X_test = last_5_rows[['Goa', 'Gujarat', 'Maharashtra', 'DNH', 'Rajasthan']]

y_true = last_5_rows['total_electricity_consumption']

# Use the model to predict electricity consumption for the last 5 rows
y_pred = model.predict(X_test)

for i in range(5):
    print(f"True Consumption: {y_true.iloc[i]} kWh")
    print(f"Predicted Consumption: {y_pred[i][0]} kWh")
    print()

1/1 [==============================] - 0s 49ms/step
True Consumption: 983.9000000000001 kWh
Predicted Consumption: 983.869384765625 kWh

True Consumption: 888.3 kWh
Predicted Consumption: 888.3291625976562 kWh

True Consumption: 933.1 kWh
Predicted Consumption: 933.113525390625 kWh

True Consumption: 945.3000000000001 kWh
Predicted Consumption: 945.2976684570312 kWh

True Consumption: 942.7 kWh
Predicted Consumption: 942.672607421875 kWh



In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true, y_pred)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_true, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Mean Absolute Error (MAE): 0.02
Mean Squared Error (MSE): 0.00
Root Mean Squared Error (RMSE): 0.02
